In [ ]:
'''
Coding: PYTHON UTF-8
Created On: 2023-04-28 12:39:30
Author: Putu Hendra Widyadharma
=== filter .pha using wadati result from makeWADATI 
'''

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from localfunction import *
pd.options.mode.chained_assignment = None

In [12]:
path = 'G:\\My Drive\\Tomography\\270423\\'
fname = 'phase_sul_2022_8P_fix.dat'
fname2 = 'temppha_phase_sul_2022_8P_fix.txt'
fname3 = 'tempwadati_phase_sul_2022_8P_fix.txt'
deviation = 9.5

In [3]:
df = readabsolute(path+fname)
dfphase = pd.read_csv(path+fname2,delim_whitespace=True)
dfwadati = pd.read_csv(path+fname3,delim_whitespace=True)

In [4]:
# hitung dan buat persamaan pembatas 
slope, intercept = np.polyfit(dfwadati['tp'].astype(float),dfwadati['ts-tp'].astype(float),1)

# predict function
predict = np.poly1d([slope, intercept])
# predict with deviation
predictup = np.poly1d([slope,intercept+deviation])
predictdown = np.poly1d([slope,intercept-deviation])

# hitung R^2 menguji kecocokan regresi
r2 = r2_score(dfwadati['ts-tp'],predict(dfwadati['tp']))
print('VP/VS = ' + str(slope + 1))
print(f"R-square = {r2}")

VP/VS = 1.7665616072517856
R-square = 0.9766163223357679


In [5]:
### dfwadati processing
# fungsi seleksi data berdasarkan batas atas dan bawah deviasi
def selection_data(row):
    if (row['ts-tp'] <= predictup(row['tp'])) and (row['ts-tp'] >= predictdown(row['tp'])):
        return True
    else:
        return False
# apply perhitungan
dfwadati['result'] = dfwadati.apply(selection_data,axis=1)
# select wadati hanya pada event yang perlu diubah
dfwadati = dfwadati[dfwadati['event_id'].isin(dfwadati[dfwadati['result'] == False]['event_id'].unique())]

In [6]:
# index search untuk event_id yang perlu disesuaikan menurut wadati 
editable_idx = df[df[0] == '#'][14].astype('int64').isin(dfwadati['event_id'].unique())
editable_idx = editable_idx[editable_idx==True].index
editable_idx

Int64Index([ 18148,  47034,  50869,  57531,  57999,  58444,  64086,  66401,
             71386,  71412,  72095,  73765,  75677,  75719,  76167,  79983,
             84743,  94460,  94750, 106661, 110447, 111937, 114556, 114646,
            115600, 115996, 127619, 131970, 135503, 137723, 141174, 144082,
            150760, 154497, 157527, 160023, 171976, 173438, 177531, 177748,
            182298, 183444, 185567, 185974, 186163, 198284, 199972, 202718,
            203006, 209298, 209457, 209965, 213667, 214066, 222628, 224955,
            226873, 229072, 229858, 238463, 240177],
           dtype='int64')

In [7]:
# buat tempat data hasil
df_result = pd.DataFrame([],columns = df.columns)

# pha index iterable
idx = df[df[0] == '#'].index

for idx_ in range (len(idx)):
# for idx_ in [1099]:
# for idx_ in [0,1,2,3]:
    # buat data per kejadian gempa
    if idx_ == len(idx)-1:
        tempdf = df.iloc[idx[idx_]::]
    else:
        tempdf = df.iloc[idx[idx_]:idx[idx_+1],:]
    # cek index header terhadap editable_idx
    if tempdf.index[0] in editable_idx:
        # proses delete data fasa S stasiun yang ts-tp diluar batas deviasi
        for i, row_ in tempdf.iterrows():
            # cari fasa S dan list stasiun yang masuk dalam membuat hasil wadati buruk 
            if row_[3]=='S' and row_[0] in (dfwadati[(dfwadati['event_id'] == int(tempdf.iloc[0,14])) & (dfwadati['result'] == False)]['sta'].to_list()):
                print(f'event: {tempdf.iloc[0,14]} -- index:{row_.name} --> {row_[0]} Phase:{row_[3]} dropped')
                tempdf.drop(row_.name,inplace=True)
    else:
        pass
    # masukkan hasil ke df hasil
    df_result = pd.concat([df_result,tempdf])

event: 1100 -- index:18150 --> TMSI Phase:S dropped
event: 2869 -- index:47041 --> SGSI Phase:S dropped
event: 3094 -- index:50886 --> KMSI Phase:S dropped
event: 3449 -- index:57542 --> SPSI Phase:S dropped
event: 3476 -- index:58007 --> PCI Phase:S dropped
event: 3501 -- index:58452 --> SPSI Phase:S dropped
event: 3813 -- index:64093 --> PMSI Phase:S dropped
event: 3938 -- index:66403 --> TNTI Phase:S dropped
event: 4205 -- index:71396 --> LUWI Phase:S dropped
event: 4207 -- index:71416 --> KMSI Phase:S dropped
event: 4207 -- index:71422 --> SRSI Phase:S dropped
event: 4207 -- index:71424 --> TOLI2 Phase:S dropped
event: 4240 -- index:72099 --> MRSI Phase:S dropped
event: 4240 -- index:72101 --> LUWI Phase:S dropped
event: 4328 -- index:73769 --> TMSI Phase:S dropped
event: 4423 -- index:75686 --> PSSI Phase:S dropped
event: 4426 -- index:75721 --> KMSI Phase:S dropped
event: 4444 -- index:76185 --> LUWI Phase:S dropped
event: 4637 -- index:79989 --> PSSI Phase:S dropped
event: 4881 

In [8]:
# perbaiki df_result dan simpan dalam bentuk pha
df_result.sort_index(inplace = True)
df_result.reset_index(inplace = True, drop = True)
df2dat(df_result,evnum = 1, path = path, fname=Path(fname).stem+'_wadatifilter.dat')
print("== data filter")
readeventphase(path+Path(fname).stem+'_wadatifilter.dat')

Output finish: temppha_phase_sul_2022_8P_fix_wadatifilter.dat at G:\My Drive\Tomography\270423\
== data filter
! temppha_phase_sul_2022_8P_fix_wadatifilter.dat
---------------------------
start time        : 2017-01-02 17:16:39.400000
end time          : 2022-12-31 22:21:43.900000
minimum magnitude : 2.5
maximum magnitude : 7.39
minimum depth     : 1 km
maximum depth     : 750 km
station used      : 137
---------------------------
Event :  10421
Phase : 232646
-- P  : 192121
-- S  :  40226
-- Pg :    155
-- Sg :    119
-- Pn :     18
-- pP :      7
---------------------------
